# Integration of CWE ID labels
* Generate new cwe_id label which integrate the cwe id labels of all three dataset

In [1]:
import numpy as np
import pandas as pd

In [2]:
def concat_to_df(df):
    df_temp = []
    for chunk in df:
        df_temp.append(chunk)

    df = pd.concat(df_temp,ignore_index = True)
    return df

In [3]:
msr_df = pd.read_csv('preprocessed_datasets/MSR_new.csv', chunksize=10000, index_col=0)

In [4]:
mvd_df = pd.read_csv('preprocessed_datasets/MVD.csv', chunksize=10000, index_col=0)

In [5]:
cfd_df = pd.read_csv('preprocessed_datasets/CFD_cwe_id.csv', index_col=0)

In [6]:
msr_df = concat_to_df(msr_df)
print(msr_df.shape)
msr_df.head(5)

(186519, 7)


,CWE ID,vul,func_before,func_after,vul_func_with_fix,lang,cwe_id
0,CWE-264,0,static bool check_rodc_critical_attribute(stru...,static bool check_rodc_critical_attribute(stru...,static bool check_rodc_critical_attribute(stru...,C,non-vulnerable
1,CWE-264,0,static int samldb_add_entry(struct samldb_ctx ...,static int samldb_add_entry(struct samldb_ctx ...,static int samldb_add_entry(struct samldb_ctx ...,C,non-vulnerable
2,CWE-264,0,static int samldb_add_entry_callback(struct ld...,static int samldb_add_entry_callback(struct ld...,static int samldb_add_entry_callback(struct ld...,C,non-vulnerable
3,CWE-264,0,static int samldb_add_handle_msDS_IntId(struct...,static int samldb_add_handle_msDS_IntId(struct...,static int samldb_add_handle_msDS_IntId(struct...,C,non-vulnerable
4,CWE-264,0,static int samldb_add_step(struct samldb_ctx *...,static int samldb_add_step(struct samldb_ctx *...,static int samldb_add_step(struct samldb_ctx *...,C,non-vulnerable


In [7]:
mvd_df = concat_to_df(mvd_df)
print(mvd_df.shape)
mvd_df.head(5)

(181641, 5)


,info,cwe_id,mvd_label,code,vul
0,150200/dirent_uri.c memset 405,non-vulnerable,0,const char *svn_relpath__internal_style(const ...,0
1,79848/CWE134_Uncontrolled_Format_String__char...,non-vulnerable,0,static void goodG2B() 64\nchar * data ; 66\nch...,0
2,94203/CWE319_Cleartext_Tx_Sensitive_Info__w32...,non-vulnerable,0,static void goodB2G2() 289\nwchar_t * password...,0
3,151133/utils.c strcmp 737,non-vulnerable,0,"int avformat_open_input(AVFormatContext **ps,c...",0
4,94266/CWE319_Cleartext_Tx_Sensitive_Info__w32...,CWE-311,32,void CWE319_Cleartext_Tx_Sensitive_Info__w32_w...,1


In [8]:
print(cfd_df.size)
cfd_df.head(5)

7909


,cwe_id
0,CWE-252
1,NVD-CWE-Other
2,CWE-20
3,CWE-119
4,CWE-19


### Check overlapped dataset

In [9]:
def normalize_cwe(cwe_list):
    normalized_cwe_list = []
    for cwe in cwe_list:
        #split the CWEs that contain multiple entries
        multiple_cwe = cwe.split(',')
        for single_cwe in multiple_cwe:
            single_cwe = single_cwe.strip() #remove leading/trailing whitespaces
            if '-' in single_cwe:
                cwe_prefix, cwe_number = single_cwe.split('-')
                # remove leading zeroes
                cwe_number = str(int(cwe_number))
                normalized_cwe = cwe_prefix + '-' + cwe_number
            else:
                normalized_cwe = single_cwe
            normalized_cwe_list.append(normalized_cwe)
    return list(set(normalized_cwe_list))

In [10]:
msr_cwe_list = msr_df['cwe_id'].unique().tolist()
print(len(msr_cwe_list))
msr_df['cwe_id'].unique()

89


array(['non-vulnerable', 'CWE-264', 'CWE-119', 'CWE-310', 'CWE-189',
       'CWE-200', 'CWE-20', 'CWE-416', 'CWE-399', 'CWE-835', 'CWE-125',
       'CWE-347', 'CWE-190', 'CWE-17', 'CWE-362', 'CWE-704', 'CWE-354',
       'CWE-209', 'CWE-78', 'CWE-59', 'CWE-388', 'CWE-617', 'CWE-476',
       'CWE-787', 'CWE-682', 'CWE-754', 'CWE-502', 'CWE-22', 'CWE-665',
       'CWE-254', 'CWE-772', 'CWE-400', 'CWE-369', 'CWE-284', 'CWE-345',
       'CWE-674', 'CWE-311', 'CWE-330', 'CWE-327', 'CWE-320', 'CWE-94',
       'CWE-415', 'CWE-134', 'CWE-732', 'CWE-331', 'CWE-287', 'CWE-16',
       'CWE-89', 'CWE-255', 'CWE-18', 'CWE-19', 'CWE-285', 'CWE-74',
       'CWE-79', 'CWE-862', 'CWE-269', 'CWE-834', 'CWE-494', 'CWE-770',
       'CWE-755', 'CWE-129', 'CWE-404', 'CWE-191', 'CWE-522', 'CWE-918',
       'CWE-358', 'CWE-295', 'CWE-346', 'CWE-532', 'CWE-426', 'CWE-77',
       'CWE-824', 'CWE-693', 'CWE-352', 'CWE-436', 'CWE-90', 'CWE-290',
       'CWE-763', 'CWE-120', 'CWE-611', 'CWE-601', 'CWE-361', 'CWE-28

In [11]:
mvd_cwe_list = mvd_df['cwe_id'].unique().tolist()
print(len(mvd_cwe_list))
mvd_df['cwe_id'].unique()

41


array(['non-vulnerable', 'CWE-311', 'CWE-706', 'CWE-400, CWE-665, CWE-20',
       'CWE-704', 'CWE-666', 'CWE-119', 'CWE-404', 'CWE-400', 'CWE-369',
       'CWE-74', 'CWE-506', 'CWE-362', 'CWE-662', 'CWE-476', 'CWE-20',
       'CWE-758', 'CWE-327', 'CWE-668', 'CWE-400, CWE-404',
       'CWE-119, CWE-666, CWE-573', 'CWE-190', 'CWE-467', 'CWE-170',
       'CWE-676', 'CWE-187', 'CWE-670', 'CWE-573', 'CWE-191', 'CWE-754',
       'CWE-666, CWE-573', 'CWE-404, CWE-668', 'CWE-400, CWE-665',
       'CWE-221', 'CWE-610', 'CWE-469', 'CWE-662, CWE-573', 'CWE-665',
       'CWE-138', 'CWE-834', 'CWE-673'], dtype=object)

In [12]:
cfd_cwe_list = cfd_df['cwe_id'].unique().tolist()
print(len(cfd_cwe_list))
cfd_df['cwe_id'].unique()

209


array(['CWE-252', 'NVD-CWE-Other', 'CWE-20', 'CWE-119', 'CWE-19',
       'CWE-94', 'CWE-255', 'CWE-189', 'CWE-79', 'CWE-264',
       'NVD-CWE-noinfo', 'CWE-120', 'CWE-399', 'CWE-362', 'CWE-193',
       'CWE-310', 'CWE-200', 'CWE-787', 'CWE-772', 'CWE-134', 'CWE-22',
       'CWE-476', 'CWE-400', 'CWE-59', 'CWE-89', 'CWE-665', 'CWE-434',
       'CWE-835', 'CWE-190', 'CWE-369', 'CWE-16', 'CWE-287', 'CWE-78',
       'CWE-352', 'CWE-640', 'CWE-284', 'CWE-330', 'CWE-276', 'CWE-611',
       'CWE-269', 'CWE-275', 'CWE-502', 'CWE-668', 'CWE-74', 'CWE-77',
       'CWE-125', 'CWE-916', 'CWE-416', 'CWE-732', 'CWE-754', 'CWE-918',
       'CWE-415', 'CWE-17', 'CWE-191', 'CWE-704', 'CWE-254', 'CWE-203',
       'CWE-116', 'CWE-273', 'CWE-295', 'CWE-763', 'CWE-18', 'CWE-681',
       'CWE-601', 'CWE-426', 'CWE-444', 'CWE-534', 'CWE-90', 'CWE-617',
       'CWE-521', 'CWE-199', 'CWE-331', 'CWE-93', 'CWE-327', 'CWE-384',
       'CWE-908', 'CWE-345', 'CWE-843', 'CWE-347', 'CWE-361', 'CWE-320',
       'CWE-3

In [13]:
if 'NVD-CWE-Other' in cfd_cwe_list:
    cfd_cwe_list.remove('NVD-CWE-Other')
if 'NVD-CWE-noinfo' in cfd_cwe_list:
    cfd_cwe_list.remove('NVD-CWE-noinfo')
print(cfd_cwe_list)

['CWE-252', 'CWE-20', 'CWE-119', 'CWE-19', 'CWE-94', 'CWE-255', 'CWE-189', 'CWE-79', 'CWE-264', 'CWE-120', 'CWE-399', 'CWE-362', 'CWE-193', 'CWE-310', 'CWE-200', 'CWE-787', 'CWE-772', 'CWE-134', 'CWE-22', 'CWE-476', 'CWE-400', 'CWE-59', 'CWE-89', 'CWE-665', 'CWE-434', 'CWE-835', 'CWE-190', 'CWE-369', 'CWE-16', 'CWE-287', 'CWE-78', 'CWE-352', 'CWE-640', 'CWE-284', 'CWE-330', 'CWE-276', 'CWE-611', 'CWE-269', 'CWE-275', 'CWE-502', 'CWE-668', 'CWE-74', 'CWE-77', 'CWE-125', 'CWE-916', 'CWE-416', 'CWE-732', 'CWE-754', 'CWE-918', 'CWE-415', 'CWE-17', 'CWE-191', 'CWE-704', 'CWE-254', 'CWE-203', 'CWE-116', 'CWE-273', 'CWE-295', 'CWE-763', 'CWE-18', 'CWE-681', 'CWE-601', 'CWE-426', 'CWE-444', 'CWE-534', 'CWE-90', 'CWE-617', 'CWE-521', 'CWE-199', 'CWE-331', 'CWE-93', 'CWE-327', 'CWE-384', 'CWE-908', 'CWE-345', 'CWE-843', 'CWE-347', 'CWE-361', 'CWE-320', 'CWE-358', 'CWE-798', 'CWE-21', 'CWE-335', 'CWE-184', 'CWE-285', 'CWE-326', 'CWE-388', 'CWE-532', 'CWE-863', 'CWE-862', 'CWE-346', 'CWE-755', 'CW

In [14]:
total_cwe_id_list = msr_cwe_list + mvd_cwe_list + cfd_cwe_list
print("Before removing overlapped cwe ids: ",len(total_cwe_id_list))
total_cwe_id_list = list(set(total_cwe_id_list))
print("After removing overlapped cwe ids: ",len(total_cwe_id_list))

Before removing overlapped cwe ids:  337
After removing overlapped cwe ids:  230


In [15]:
'non-vulnerable' in total_cwe_id_list

True

In [16]:
# Check NaN
'NaN' in total_cwe_id_list

False

### Define integrated cwe_ids labels

In [17]:
# sorted by cwe_ids and non-vulnerable as index: 0

import re

def sort_by_number(my_list):
    def extract_number(string):
        match = re.search(r'\d+', string)
        if match:
            return int(match.group())
        else:
            return float('inf')
    return sorted(my_list, key=extract_number)

total_cwe_id_list = sort_by_number(total_cwe_id_list)
total_cwe_id_list.remove('non-vulnerable')
total_cwe_id_list.insert(0, 'non-vulnerable')
print(total_cwe_id_list)  # prints ['item1', 'item2', 'item3', 'item10', 'item25']


['non-vulnerable', 'CWE-16', 'CWE-17', 'CWE-18', 'CWE-19', 'CWE-20', 'CWE-21', 'CWE-22', 'CWE-23', 'CWE-27', 'CWE-59', 'CWE-73', 'CWE-74', 'CWE-77', 'CWE-78', 'CWE-79', 'CWE-80', 'CWE-87', 'CWE-88', 'CWE-89', 'CWE-90', 'CWE-91', 'CWE-93', 'CWE-94', 'CWE-98', 'CWE-113', 'CWE-114', 'CWE-116', 'CWE-117', 'CWE-119', 'CWE-119, CWE-666, CWE-573', 'CWE-120', 'CWE-121', 'CWE-122', 'CWE-125', 'CWE-126', 'CWE-129', 'CWE-131', 'CWE-134', 'CWE-138', 'CWE-170', 'CWE-172', 'CWE-178', 'CWE-184', 'CWE-185', 'CWE-187', 'CWE-189', 'CWE-190', 'CWE-191', 'CWE-193', 'CWE-199', 'CWE-200', 'CWE-203', 'CWE-208', 'CWE-209', 'CWE-212', 'CWE-221', 'CWE-241', 'CWE-248', 'CWE-250', 'CWE-252', 'CWE-254', 'CWE-255', 'CWE-264', 'CWE-266', 'CWE-269', 'CWE-273', 'CWE-275', 'CWE-276', 'CWE-277', 'CWE-279', 'CWE-281', 'CWE-284', 'CWE-285', 'CWE-287', 'CWE-288', 'CWE-290', 'CWE-294', 'CWE-295', 'CWE-297', 'CWE-303', 'CWE-304', 'CWE-305', 'CWE-306', 'CWE-307', 'CWE-310', 'CWE-311', 'CWE-312', 'CWE-319', 'CWE-320', 'CWE-321

In [18]:
total_cwe_dict = { cwe_id_key: idx for idx, cwe_id_key in enumerate(total_cwe_id_list)}
print(total_cwe_dict)

{'non-vulnerable': 0, 'CWE-16': 1, 'CWE-17': 2, 'CWE-18': 3, 'CWE-19': 4, 'CWE-20': 5, 'CWE-21': 6, 'CWE-22': 7, 'CWE-23': 8, 'CWE-27': 9, 'CWE-59': 10, 'CWE-73': 11, 'CWE-74': 12, 'CWE-77': 13, 'CWE-78': 14, 'CWE-79': 15, 'CWE-80': 16, 'CWE-87': 17, 'CWE-88': 18, 'CWE-89': 19, 'CWE-90': 20, 'CWE-91': 21, 'CWE-93': 22, 'CWE-94': 23, 'CWE-98': 24, 'CWE-113': 25, 'CWE-114': 26, 'CWE-116': 27, 'CWE-117': 28, 'CWE-119': 29, 'CWE-119, CWE-666, CWE-573': 30, 'CWE-120': 31, 'CWE-121': 32, 'CWE-122': 33, 'CWE-125': 34, 'CWE-126': 35, 'CWE-129': 36, 'CWE-131': 37, 'CWE-134': 38, 'CWE-138': 39, 'CWE-170': 40, 'CWE-172': 41, 'CWE-178': 42, 'CWE-184': 43, 'CWE-185': 44, 'CWE-187': 45, 'CWE-189': 46, 'CWE-190': 47, 'CWE-191': 48, 'CWE-193': 49, 'CWE-199': 50, 'CWE-200': 51, 'CWE-203': 52, 'CWE-208': 53, 'CWE-209': 54, 'CWE-212': 55, 'CWE-221': 56, 'CWE-241': 57, 'CWE-248': 58, 'CWE-250': 59, 'CWE-252': 60, 'CWE-254': 61, 'CWE-255': 62, 'CWE-264': 63, 'CWE-266': 64, 'CWE-269': 65, 'CWE-273': 66, 'CW

In [19]:
# save dict for later retrieval
import pickle

with open("preprocessed_datasets/total_cwe_dict.txt", "wb") as myFile:
    pickle.dump(total_cwe_dict, myFile)

In [20]:
def check_match_vul_and_CWE_ID(df,cwe_id_col_name):
    count = 0
    for index, row in df.iterrows():
        if row['vul'] == 0 and row[cwe_id_col_name] != 'non-vulnerable':
            print("vul and CWE ID are not matched!!!!!",row['vul'], row['CWE ID'])
            break

    if count==0:
        print("All vul and CWE ID are matched correctly")
        
def check_match_vul_and_label(df):
    count = 0
   
    for index, row in df.iterrows():
        if row['vul'] == 0 and row['label'] != 0:
            print("vul and label are not matched!!!!!",row['vul'], row['label'])
            break
    if count==0:
        print("All vul and label are matched correctly")


            

In [21]:
check_match_vul_and_CWE_ID(msr_df,'cwe_id')

All vul and CWE ID are matched correctly


In [22]:
check_match_vul_and_CWE_ID(mvd_df,'cwe_id')

All vul and CWE ID are matched correctly


In [23]:
# add new column based on dictionary values
mvd_df['label'] = mvd_df['cwe_id'].apply(lambda x: total_cwe_dict.get(x))

mvd_pre_df = mvd_df[['code', 'vul', 'cwe_id','label']]
mvd_pre_df.head(5)

,code,vul,cwe_id,label
0,const char *svn_relpath__internal_style(const ...,0,non-vulnerable,0
1,static void goodG2B() 64\nchar * data ; 66\nch...,0,non-vulnerable,0
2,static void goodB2G2() 289\nwchar_t * password...,0,non-vulnerable,0
3,"int avformat_open_input(AVFormatContext **ps,c...",0,non-vulnerable,0
4,void CWE319_Cleartext_Tx_Sensitive_Info__w32_w...,1,CWE-311,86


In [26]:
# add new column based on dictionary values
msr_df['label'] = msr_df['cwe_id'].apply(lambda x: total_cwe_dict.get(x))

msr_pre_df = msr_df[['func_before', 'vul', 'cwe_id','label']]
msr_pre_df = msr_pre_df.rename(columns={'func_before':'code'})
msr_pre_df.head(5)

,code,vul,cwe_id,label
0,static bool check_rodc_critical_attribute(stru...,0,non-vulnerable,0
1,static int samldb_add_entry(struct samldb_ctx ...,0,non-vulnerable,0
2,static int samldb_add_entry_callback(struct ld...,0,non-vulnerable,0
3,static int samldb_add_handle_msDS_IntId(struct...,0,non-vulnerable,0
4,static int samldb_add_step(struct samldb_ctx *...,0,non-vulnerable,0


In [27]:
check_match_vul_and_label(mvd_pre_df)

All vul and label are matched correctly


In [28]:
check_match_vul_and_label(mvd_pre_df)

All vul and label are matched correctly


In [29]:
# save it 
# concatenated_df.to_csv('preprocessed datasets/mvd_msr.csv')

msr_pre_df.to_csv('preprocessed_datasets/MSR_labeled.csv')
mvd_pre_df.to_csv('preprocessed_datasets/MVD_labeled.csv')